In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import pickle
import datetime
import logging
from CNN import malware_RNN
from config import get_config
from sklearn.model_selection import train_test_split
from tensorflow.contrib import learn
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import accuracy_score

logging.getLogger().setLevel(logging.INFO)

In [2]:
#Data parameters
tf.flags.DEFINE_string("data_path", "/home/zx/gram3_gain1000/data_process_data_original_value_test","data_path")
tf.flags.DEFINE_string("save_path" , "/home/zx/RCNN/CNN/session_save/","Model output directory.")
tf.flags.DEFINE_string("checkpoint_dir","/home/zx/RCNN/CNN/session_save/checkpoints","checkpoint directory from train run")
#Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement",True,"Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement",False,"Log placement of ops on devices")
FLAGS =tf.flags.FLAGS

In [3]:
def load_data(file_name):
    with open(file_name, "rb") as f:
        raw_x, raw_y = pickle.load(f)
    return raw_x,raw_y
def real_len_test(batches):
    real=[]
    for batch in batches:
        if min(batch)>=1:
            result=1000
        else:
            result=np.argmin(batch)
        real.append(result) 
    return real

In [4]:
def test():
    x_test,y_test=load_data(FLAGS.data_path)
    checkpoint_file=tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
    print(checkpoint_file)
    graph=tf.Graph()
    with graph.as_default():
        session_conf=tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement,log_device_placement=FLAGS.log_device_placement)
        sess=tf.Session(config=session_conf)
        with sess.as_default():
            print(checkpoint_file)
            #Load the saved meta graph and restore variables
            saver=tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
            saver.restore(sess,checkpoint_file)
            #get the placeholder from the graph by name
            input_x=graph.get_operation_by_name("input_x").outputs[0]
            input_y=graph.get_operation_by_name("input_y").outputs[0]
            dropout_keep_prob=graph.get_operation_by_name("dropout_keep_prob").outputs[0]
            real_len=graph.get_operation_by_name("real_len").outputs[0]
            batch_size=graph.get_operation_by_name("batch_size").outputs[0]
            y_pre=graph.get_operation_by_name("loss/y_pred").outputs[0]
            #predictions=graph.get_operation_by_name("output/predictions")
            #accuracy=graph.get_operation_by_name("accuracy/accuracy")
            #get the predictions value
                #feed,give
            feed_dict={
            input_x:x_test,
            input_y:y_test,
            dropout_keep_prob:1.0,
            batch_size:len(x_test),
            real_len:real_len_test(x_test)
              }
            #out
            fetches = {
            "y_pre" :y_pre
            }
            fetches_ret=sess.run(fetches , feed_dict)
            y_p=fetches_ret[y_pre]
            auc=roc_auc_score(y_test, y_p)
            for i,y_acc in enumerate(y_p):
                if y_acc>0.5:
                    y_p[i]=1
                else:
                    y_p[i]=0
            acc=accuracy_score(y_test, y_p)
            logging.critical("auc:{}".format(auc))

        
        
        
            
            
            

In [5]:
def main(argv=None):
    if not FLAGS.data_path:
        raise ValueError("Must set --data_path to data file")
    test()
    

In [6]:
if __name__ == '__main__':
    main()

/home/zx/RCNN/CNN/session_save/checkpoints/model-14
/home/zx/RCNN/CNN/session_save/checkpoints/model-14


NameError: global name 'y_p' is not defined

No traceback available to show.
